# Modeling a Pin-Cell
In this module, we'll demonstrate the basic features of the Python API for constructing input files and running OpenMC. In it, we will show how to create a basic reflective pin-cell model that is equivalent to modeling an infinite array of fuel pins. We highly recommend having a copy of the [Python API reference documentation](https://docs.openmc.org/en/stable/pythonapi/index.html) open in another browser tab that you can refer to.

For this example, we'll create a pin-cell of the CEFR fuel assembly with simplified material compositions. More accurate material compositions will be applied as part of the exercise at the end of the day.

- UO<sub>2</sub>
- Helium
- Sodium
- Stainless Steel

all at a temperature of 523.15 K.

The dimensions of our fuel pin will be as follows:
- Fuel pin center hold radius = 0.08020 cm
- Fuel outer radius = 0.0.25565 cm
- Clad inner radius = 0.27112 cm
- Clad outer radius = 0.30499 cm
- Fuel pin pitch = 0.695 cm

In [ ]:
import openmc

In [ ]:
# remote any existing OpenMC input XMl files and output HDF5 files
!rm *.xml *.h5

## Naming Conventions

Before we start working with the Python API, it's helpful to understand the naming convention of objects so that you can conceptualize what is a function, what is a class, etc. OpenMC's Python interface follows the same naming convention that is adopted by many/most Python projects:

- Module names are `lowercase`
- Functions are `lowercase_with_underscores`
- Classes are `CamelCase`
- Class attributes/variables are `lowercase_with_underscores`
- Top-level (global) variables are `UPPERCASE_WITH_UNDERSCORES`

To give a few specific examples:

- `openmc.deplete` is the depletion _module_
- `openmc.run` is a _function_
- `openmc.Material` is a _class_
- `openmc.data.ATOMIC_NUMBER` is a _top-level variable_

## Setting Attributes

When building OpenMC models, we will work with many different classes. Each class typically has _attributes_, a variable belonging to the class. When creating a class, you can often set those attributes directly when you instantiate the object, like this:

Or, you can assign values to attributes after you have already created the object. For example, the following is equivalent to the above:

You can in general also mix-and-match, setting some attributes when you instantiate the object, and others at a later point.


## How to Get Help/Learn More

When building OpenMC models, we recommend having a copy of the Python API documentation (linked earlier) open. You can also query information about classes and methods directly from Jupyter. For example, we can use `help` to get documentation on all of the valid attributes for functions and classes.

In [ ]:
help(openmc.Cell)

Attributes which have a default value will appear as `<attribute>=<the default value>` in the class/function definition. For classes, all of the member functions you can access will then be defined further down, with their associated documentation. To see documentation for just one specific function, you can use syntax like:

In [ ]:
my_cell = openmc.Cell()
help(my_cell.from_xml_element)

You can also query the type of a particular object using `type`. Note that we can also display the attributes for a particular object using `print`.

In [ ]:
type(my_cell)
print(my_cell)

## Defining Materials

Materials in OpenMC are defined as a set of nuclides with specified atom/weight fractions. To begin, we will create a material by making an instance of the `Material` class. In OpenMC, many objects, including materials, are identified by a "unique ID" that is simply just a positive integer. These IDs are used when exporting XML files that the solver reads in. They also appear in the output and can be used for identification. Since an integer ID is not very useful by itself, you can also give a material a `name` as well.

If you don't give an ID, one will be automatically generated for you:

We see that an ID of 2 was automatically assigned. Let's now move on to adding nuclides to our `uo2` material. The `Material` object has a method `add_nuclide()` whose first argument is the name of the nuclide and second argument is the atom or weight fraction.

In [ ]:
help(uo2.add_nuclide)

We see that by default it assumes we want an atom fraction.

Now we need to assign a total density to the material. We'll use the `set_density` for this.

You may sometimes be given a material specification where all the nuclide densities are in units of atom/b-cm. In this case, you just want the density to be the sum of the constituents. In that case, you can simply run `mat.set_density('sum')`.

With UO2 finished, let's now create materials for the clad and coolant. Note the use of `add_element()` for zirconium.

### Element Expansion

Did you notice something really cool that happened to our Fe and C elements? OpenMC automatically turned it into a list of nuclides when it exported it! The way this feature works is as follows:

- First, it checks whether `Materials.cross_sections` has been set, indicating the path to a `cross_sections.xml` file.
- If `Materials.cross_sections` isn't set, it looks for the `OPENMC_CROSS_SECTIONS` environment variable.
- If either of these is found, it scans the file to see what nuclides are actually available and will expand elements accordingly.

---
## An aside on S($\alpha$, $\beta$) tables

For systems with a significant thermal component to the neutron flux, it is important to augment the free atom cross sections we've used up to this point when possible. This can be done by using an $S(\alpha,\beta)$ table so that the bound atom cross section is used at thermal energies. To do this, there's an `add_s_alpha_beta()` method. Note the use of the GND-style name "c_H_in_H2O".

```python
water.add_s_alpha_beta('c_H_in_H2O')
```

---

When we go to run the transport solver in OpenMC, it is going to look for a `materials.xml` file. Thus far, we have only created objects in memory. To actually create a `materials.xml` file, we need to instantiate a `Materials` collection and export it to XML.

Note that `Materials` is actually a subclass of Python's built-in `list`, so we can use methods like `append()`, `insert()`, `pop()`, etc.

Finally, we can create the XML file with the `export_to_xml()` method. In a Jupyter notebook, we can run a shell command by putting `!` before it, so in this case we are going to display the `materials.xml` file that we created.

We see that now He3 and He4 were automatically added. Other isotopes are missing because our cross sections file (which is based on ENDF/B-VII.1) doesn't contain data for them. If OpenMC didn't know about the cross sections file, it would have assumed that all isotopes exist.

### Materials from a chemical formula

### Material mixtures

### The `cross_sections.xml` file

The `cross_sections.xml` tells OpenMC where it can find nuclide cross sections and $S(\alpha,\beta)$ tables. It serves the same purpose as MCNP's `xsdir` file and Serpent's `xsdata` file. As we mentioned, this can be set either by the `OPENMC_CROSS_SECTIONS` environment variable or the `Materials.cross_sections` attribute.

Let's have a look at what's inside this file:

The cross section dataset used by OpenMC can also be configured using the `openmc.config` property of the `openmc` module. Setting this property inside a Python interpreter session is equivalent to updating the cross sections environment variable discussed above.

In [ ]:
# openmc.config['cross_sections'] = /path/to/my/xs/data/cross_sections.xml

### Enrichment

Note that the `add_element()` method has a special argument `enrichment` that can be used for Uranium. For example, if we know that we want to create 3% enriched UO2, the following would work:

---

## Defining Geometry

At this point, we have three materials defined, exported to XML, and ready to be used in our model. To finish our model, we need to define the geometric arrangement of materials. OpenMC represents physical volumes using constructive solid geometry (CSG), also known as combinatorial geometry. The object that allows us to assign a material to a region of space is called a `Cell` (same concept in MCNP, for those familiar). There are four major stages in building a cell:

### Surfaces
In order to define a region that we can assign to a cell, we must first define surfaces which bound the region. A *surface* is a locus of zeros of a function of Cartesian coordinates $x$, $y$, and $z$, e.g.

- A plane perpendicular to the x axis: $x - x_0 = 0$
- A cylinder parallel to the z axis: $(x - x_0)^2 + (y - y_0)^2 - R^2 = 0$
- A sphere: $(x - x_0)^2 + (y - y_0)^2 + (z - z_0)^2 - R^2 = 0$

Between those three classes of surfaces (planes, cylinders, spheres), one can construct a wide variety of models. It is also possible to define cones and general second-order surfaces.

### Half-Spaces
Note that defining a surface is not sufficient to specify a volume -- in order to define an actual volume, one must reference the half-space of a surface. A surface *half-space* is the region whose points satisfy a positive or negative inequality of the surface equation. For example, for a sphere of radius one centered at the origin, the surface equation is $f(x,y,z) = x^2 + y^2 + z^2 - 1 = 0$. Thus, we say that the negative half-space of the sphere, is defined as the collection of points satisfying $f(x,y,z) < 0$, which one can reason is the inside of the sphere. Conversely, the positive half-space of the sphere would correspond to all points outside of the sphere.

### Regions
A region is then a combination (or just one) half-space.

### Cells

A Cell ties a region together with a "fill" (see below). Cells can be grouped together into `Universes` to produce replicated geometry (discussed in more detail later).

### Fills
Finally, a cell is complete once we have defined what is _filling_ the cell, which may be one of:

- material
- nothing (`None`), or vacuum/void
- universe
- lattice

Note that by default the sphere is centered at the origin so we didn't have to supply `x0`, `y0`, or `z0` arguments. Strictly speaking, we could have omitted `r` as well since it defaults to one. To get the negative or positive half-space, we simply need to apply the `-` or `+` unary operators, respectively.

(NOTE: Those unary operators are defined by special methods: `__pos__` and `__neg__` in this case).

Now let's see if `inside_sphere` actually contains points inside the sphere:

Everything works as expected! Now that we understand how to create half-spaces, we can create more complex volumes by combining half-spaces using Boolean operators: `&` (intersection), `|` (union), and `~` (complement):

- `&`: logical AND
- `|`: logical OR
- `~`: logical NOT

For example, let's say we want to define a region that is the top part of the sphere (all points inside the sphere that have $z > 0$.

For many regions, OpenMC can automatically determine a bounding box. To get the bounding box, we use the `bounding_box` property of a region, which returns a tuple of the lower-left and upper-right Cartesian coordinates for the bounding box:

Now that we see how to create volumes, we can use them to create a cell.

The full [list of available surfaces](https://docs.openmc.org/en/stable/pythonapi/base.html#building-geometry) is as follows.

Planes:

- `openmc.Plane` — An arbitrary plane of the form $Ax + By + Cz = D$
- `openmc.XPlane` — A plane perpendicular to the x axis of the form $x - x_0 = 0$
- `openmc.YPlane` — A plane perpendicular to the y axis of the form $y - y_0 = 0$
- `openmc.ZPlane` — A plane perpendicular to the z axis of the form $z - z_0 = 0$

Quadrics:

- `openmc.XCylinder` — An infinite cylinder whose length is parallel to the x-axis of the form $(y - y_0)^2 + (z - z_0)^2 = r^2$
- `openmc.YCylinder` — An infinite cylinder whose length is parallel to the x-axis of the form $(x - x_0)^2 + (z - z_0)^2 = r^2$
- `openmc.ZCylinder` — An infinite cylinder whose length is parallel to the x-axis of the form $(x - x_0)^2 + (y - y_0)^2 = r^2$
- `openmc.Sphere` — A sphere of the form $(x - x_0)^2 + (y - y_0)^2 + (z - z_0)^2 = r^2$
- `openmc.XCone` — A cone parallel to the x-axis of the form $(y - y_0)^2 + (z - z_0)^2 = r^2 (x - x_0)^2$
- `openmc.YCone` — A cone parallel to the y-axis of the form $(x - x_0)^2 + (z - z_0)^2 = r^2 (y - y_0)^2$
- `openmc.ZCone` — A cone parallel to the z-axis of the form $(x - x_0)^2 + (y - y_0)^2 = r^2 (z - z_0)^2$
- `openmc.Quadric` — A generic quadric surface

Torii:

- `openmc.XTorus` — A torus of the form $(x - x_0)^2/B^2 + (\sqrt{(y - y_0)^2 + (z - z_0)^2} - A)^2/C^2 - 1 = 0$
- `openmc.YTorus` — A torus of the form $(y - y_0)^2/B^2 + (\sqrt{(x - x_0)^2 + (z - z_0)^2} - A)^2/C^2 - 1 = 0$
- `openmc.ZTorus` — A torus of the form $(z - z_0)^2/B^2 + (\sqrt{(x - x_0)^2 + (y - y_0)^2} - A)^2/C^2 - 1 = 0$

### Boundary Conditions

When you create a surface, by default particles that pass through the surface will consider it to be transmissive, i.e. they pass through the surface freely. To specify boundary conditions, you simply need to set the `Surface.boundary_type` to one of:

- `vacuum`
- `reflective`
- `periodic` (either rotational or translational)
- `white`

By default, the cell is not filled by any material (void). In order to assign a material, we set the `fill` property of a `Cell`.

## Universes and in-line plotting

A collection of cells is known as a universe (again, this will be familiar to MCNP/Serpent users) and can be used as a repeatable unit when creating a model. Although we don't need it yet, the benefit of creating a universe is that we can visualize our geometry while we're creating it.

The `Universe` object has a `plot` method that will display our the universe as current constructed:

By default, the plot will appear in the $x$-$y$ plane. We can change that with the `basis` argument.

If we have particular fondness for, say, fuchsia, we can tell the `plot()` method to make our cell that color.

---

## The CEFR pincell geometry

We now have enough knowledge to create our pin-cell. For CEFR, we need four surfaces to define the fuel and clad:

1. The inner surface of the fuel hole -- a cylinder parallel to the z axis
2. The outer surface of the fuel -- same as above
3. The inner surface of the cladding -- same as above
4. The outer surface of the cladding -- same as above

These four surfaces will all be instances of `openmc.ZCylinder`, each with a different radius according to the specification.

Finally, we need to handle the coolant outside of our fuel pin. To do this, we create x- and y-planes that bound the geometry.

The sodium region is going to be everything outside of the clad outer radius and within the box formed as the intersection of four half-spaces.

The final step is to assign the cells we created to a universe and tell OpenMC that this universe is the "root" universe in our geometry. The `Geometry` is the final object that is actually exported to XML.

# Starting source and settings

The Python API has a module `openmc.stats` with various univariate and multivariate probability distributions. We can use these distributions to create a starting source using the `openmc.Source` object. One can independently specify the spatial distribution (`space`), the angular distribution (`angle`), the energy distribution (`energy`), and the time distribution (`time`). For this example, we'll only specify the spatial distribution as a single point.

Now let's create a `Settings` object and give it the source we created along with specifying how many batches and particles we want to run.

---

## Temperature Treatment

Generally, cross section data in OpenMC is specified at specific temperature values. Material or cell temperatures between these values can be used, however. OpenMC can model the temperature using a few different techniques:

  - **nearest**: OpenMC will use the nearest temperature cross section data available within some tolerance
  - **interpolate**: OpenMC will interpolate between the two nearest temperatures

[temperature treatment documentation](https://ec2-44-193-2-67.compute-1.amazonaws.com/)

For the CEFR model, we'll treat all of our materials as if they were at 523.15 K.

---

# Running OpenMC

Running OpenMC from Python can be done using the `openmc.run()` function. This function allows you to set the number of MPI processes and OpenMP threads, if need be.

Great! OpenMC already told us our k-effective.

# Geometry plotting

We saw before that we could call the `Universe.plot()` method to show a universe while we were creating our geometry. There is also a built-in plotter in the codebase that is much faster than the Python plotter and has more options. The interface looks somewhat similar to the `Universe.plot()` method. Instead though, we create `Plot` instances, assign them to a `Plots` collection, export it to XML, and then run OpenMC in geometry plotting mode. As an example, let's specify that we want the plot to be colored by material (rather than by cell) and we assign yellow to fuel and blue to water.

With our plot created, we need to add it to a `Plots` collection which can be exported to XML.

Now, we can use functionality from IPython to display the `.png` image inline in our notebook:

OpenMC also provides us with a method on the `Plot` class that simplifies the workflow.

# The `Model` class

So far, we've seen that to create and simulate a model, we had to create an instance of `Geometry`, `Materials`, and `Settings` and call the `export_to_xml` method on each of them. OpenMC also provides a `Model` class that aggregates these classes together an provides a single `export_to_xml` method that will export all files.

The `Model` class also has `run` method that will both export single XML file (`model.xml`) *and* run a simulation, returning the name of the last statepoint file that was written:

<div class="alert alert-block alert-info">
Note: If both separate XML inputs (`materials.xml`, `geometry.xml`, `settings.xml`, ...) and a `model.xml` file are both present in OpenMC's run directory, the `model.xml` fill will be used instead of the separate XML inputs.    
</div>

As we'll see later, the `Model` class has other useful features and can be very useful for postprocessing work.

---

# Basic Tallies in OpenMC

**Learning Objectives:**

  - Understand application of filters and scores to create tallies
  - Apply tallies to an OpenMC simulation
  - Extract information from OpenMC statepoint files
  - Understand tally units and normalization
  - Plot tally results

In this section, we'll be looking at how to extract custom information from an OpenMC simulation in what is known as a tally. A tally accumulates statistical information during the simulation about particles when they eneter regions of phase space specified on the tally. The limits of these regions are set by filters applied to the tally. Scores and nuclides can also be applied to tallies to indicate what type of information is kept about the particle (e.g. reaction types, flux, heat, etc.).

Any tally in OpenMC can be described with the following form:

$$ 
 X = \underbrace{\int d\mathbf{r} \int d\mathbf{\Omega} \int
    dE}_{\text{filters}} \underbrace{f(\mathbf{r}, \mathbf{\Omega},
    E)}_{\text{scores}} \psi (\mathbf{r}, \mathbf{\Omega}, E)
$$

where filters set the limits of the integrals and the scoring function is convolved with particle information (e.g. reaction type, current material, etc.).

First, we'll import a few additional packages to help manage data and visualize results.

We'll continue to work with the CEFR pincell model we've developed thus far. Let's explore that model further.

In this exercise we'll be adding tallies to perform a few different tasks:


  **1. Determine the energy and heat produced per fission** \
  **2. Plot the flux spectrum of the pincell** \
  **3. Plot reaction types based on material**
  
To do this we'll use a variety of different filters applied to different tallies. 

First, to determine the recoverable energy produced per fission we'll create a tally without filters to gather information on the fission reaction rate ("`fission`") and recoverable fission energy ("`kappa-fission`"). Because we want this information talllied throughout the model, a "global" tally, no filters need to be applied.



<div class="alert alert-block alert-info">
A full list of scores and their meanings can be found <a href=https://docs.openmc.org/en/stable/usersguide/tallies.html#scores >here</a>.
</div>

## Task 1: Energy released per fission

## [Link to tally score descriptions](https://docs.openmc.org/en/stable/usersguide/tallies.html#scores)

Similar to the `openmc.Materials` object, the `openmc.Tallies` object is a sub-class of Python's built-in `list` and is used to collect tallies for appliction to the model.

If we list our current directory, we see that several new files have been created as a result of this run: `summary.h5`, `tallies.out`, and `statepoint.50.h5`. The summary file contains information about the simulation's setup (geometry, materials, meshes, etc.) in an HDF5 format. The `tallies.out` file contains a text output of all user-specified tallies for the simulation.

This can be useful to quickly look at simple tally results, but isn't a great format to post-process simulation data. For that we'll look to the statepoint file.

To extract information from the statepoint file we'll create an `openmc.StatePoint` object. The `statepoint.get_tally` function will search for tallies by scores, filters, nuclides, ids, and return the closest match. Exact matches can be speficied as well.

If we print the tally objects returned, we see that they indeed match the tally specification we generated above.

<div class="alert alert-block alert-info">
<b>A quick aside on how statepoint objects interact with summary files:</b>


The `openmc.statepoint` object will read information from the `summary.h5` file if one is present, keeping that file open in the Python interpreter. The open `summary.h5` file can interfere with the initialization of subsequent OpenMC simulations. It is recommended that information be extracted from statepoints within a [context manager](https://book.pythontips.com/en/latest/context_managers.html) as we do here. Alternatively, making sure to call the `openmc.StatePoint.close` method will work also. For more details please look to the [relevant section in the user's guide](https://docs.openmc.org/en/stable/usersguide/troubleshoot.html#runtimeerror-failed-to-open-hdf5-file-with-mode-w-summary-h5).   
</div>



To compute the energy released per fission event, we can simply take the tallied energy released per fission and divide it by the fission rate.

The array of results returned from the `Tally.get_values` method returns an array with 3 dimensions: one for filter bins, one for nuclides, and one for the scores requested.

In [ ]:
help(openmc.Tally.get_values)

For a water reactor with U235 as the only fissioning isotope this is about what we would expect: ~193-194 MeV! Okay, on to some new tallies with filters.

As with most values coming out of an MC code, the tally values in the statepoint file are per source-particle. In this case, these units cancel out, but this will not be the case in our next example.

---

## Task 2: Plot the neutron flux spectrum


To perform this task, we'll be applying a tally with an energy filter and a score. OpenMC's data module contains different group structures. For this problem we'll use the CASMO-70 group structure. An energy filter can easily be created from a pre-defined group structure in OpenMC as follows:

Now we'll apply this tally and re-run the problem

Now to plot the spectrum

## Normalizing Tallies

Note that the units of flux in the above plot are in $\frac{particle-cm}{source-particle}$. As is the case with many values tallied by Monte Carlo codes, the value of the flux does not account for volume and is in terms of the number of source particles emitted. To generate this same plot in terms of absolute flux units ($\frac{particle}{cm^{2}-s}$) we'll need to normalize this tally by:

  - the volume of the region the tally covers
  - the number of source particle emitted

In this case, the volume of the region is the volume of the entire pincell. Because we're working with a 2-D model, we'll get units that give us the flux per unit length of the pincell in the axial direction. For simplicity, we'll assume that our pincell is 1 cm in height to make life easier. OpenMC's geometry object allows for computation of a bounding box for geometries like this that will help make this more straight forward.

Determining the number of source particles per second is more complicated, however. This means computing the eV/source particle due to fission. To get the source rate, we'll need the following pieces of information:

  1. the total power produced in the tally region (known a priori)
  2. the heat produced by fission power, per source particle
  
 To get this information we'll need to construct another tally to get additional information from the simulation.

The combination of the following tally values and power provide us with the source normalization needed as follows:


$$ \text{neutron source} [\frac{n}{s}] = \frac{1}{\text{heat per fission} [\frac{eV}{source}]} [\frac{source}{eV}]\times \frac{1}{1.6\times 10^{-19}} [\frac{eV}{J}] \times \text{power} [\frac{J}{s}]$$ 

We can now use this information to normalize our flux values and reproduce our plot in more standard units.

---

## Task 3: Reaction Types by Material

Looking at the different reaction types by material will require a material filter and the set of reaction types we want to score. For this example, we'll be scoring absorption, scattering and fission in each material.

To start, we'll create a material filter.

Now we'll gather information from the statepoint file about each score we applied to the tally. With multiple scores and materials, we'll use a Pandas data frame to view the results in a more coherent manner.

Each score has three values -- one for each material in the model.

First, we'll add a new column to the data frame with normalized results.

We'll add a new entry in the dataframe for our material names to make plotting easier.

---

# Universes

A universe is a collection of cells that can be used as a repeatable unit in the geometry. At a minimum, there must be one "root" universe (say, named `root`), which gets passed to `openmc.Geometry(root)`. But you can also use universes to repeat a collection of cells multiple times throughout a geometry. Here, we will explore some basic features of universes.

We'll start by making a universe which looks similar to the pincell we built earlier - say, a cylinder of UO2 enclosed by an infinite region of water. First, we set up our materials and create our geometry.

First, let's make a simplified pincell model to explore how universes work.

We have created a universe containing a pin, enclosed in an infinite medium of water. Now let's suppose that I want to fill this universe into an enclosing cell, a cylinder of radius 5 cm. Let's first create this cylinder, and then we will fill it with our `universe`.

Let's take a look at our geometry. In order to visualize at this stage, we need to create a universe from our `big_cell`.

We can see that our `big_cell`, the large cylinder, has been filled with the `universe` we declared earlier. Let's increase the complexity a bit to understand how this filling works. What if the cylinder of UO2 in our `universe` is not located at the origin, but is instead shifted to a different position?

We see that when we fill a universe inside of another cell, that there's (by default) no transformation of coordinates. You can shift the position of the universe filling a cell with the `Cell.translation` attribute. There are similar adjustments you can make, like rotations.

# Lattices

Lattices are a convenient way to (i) repeat a universe multiple times in space, while (ii) automatically translating that universe's origin to different positions in space.

In this exercise, we'll build two lattices, one rectangular and one hexagonal, using the CEFR pincell.

Later in this example, we will make a bunch of geometry plots.  By default, every region is colored randomly and the results are Now that we know our materials, let's define a "color specification" to use when plotting our geometry.

### An alternative approach to building a pincell universe

Initally, we built the CEFR pincell model by specifying each surface manually. There are many convenience functions in OpenMC that make this process even easier.

---

## Lattices in OpenMC

OpenMC has `RectLattice` and `HexLattice` objects.  For our fuel assembly, we need to use `RectLattice`. Let's look at a simple one before we do the full assembly.

First, we also need to define a universe that is all water.

When creating a rectangular lattice, we need to define:

1. The lower-left coordinates of the lattice (`.lower_left`)
2. The size of each lattice element (`.pitch`)
3. The 2D arrangement of universes (`.universes`)
4. (_optionally_) A universe that is used outside of the defined region (`.outer`)

## Rectangular Lattices

To plot the lattice, we need to put it in a universe. For this, we'll create a single cell filled with the lattice, and then put that single cell inside a universe to plot:

**What exactly does `outer` mean?**

In the previous section, we set the lattice outer universe to a universe containing a single cell with only water in it. To get a better sense of what the outer universe does, let's change the outer universe to the guide tube universe:

## Hexagonal Lattices

OpenMC also allows you to define hexagonal lattices. They are a little trickier, but as we'll see there are some helper methods that demystify how to assign universes.

We need to set the `center` of the lattice, the `pitch`, an `outer` universe (which is applied to all lattice elements outside of those that are defined), and a list of `universes`. Let's start with the easy ones first. Note that for a 2D lattice, we only need to specify a single number for the pitch.

Now we need to set the `universes` property on our lattice. It needs to be set to a list of lists of Universes, where each list of Universes corresponds to a ring of the lattice. The rings are ordered from outermost to innermost, and within each ring the indexing starts at the "top". To help visualize the proper indices, we can use the `show_indices()` helper method.

Let's setup a hexagonal lattice assembly like the one in the CEFR benchmark model.

Now let's put our lattice inside a circular cell.

### Rotating the lattice

Now let's say we want our hexagonal lattice orientated such that flat sides are parallel to the y-axis instead of the x-axis. This can be achieved by changing the orientation of the lattice from `'y'` to `'x'`:

Again, we can use the `show_indices` method to see the ordering of universes within each ring:

# Final CEFR Assembly Modeling Steps

The circular boundary is nice for some initial visualization, but in reality the CEFR assemblies are bounded by stainless steel ducts. Let's add those to the model now.

In the image of the CEFR assembly above, the steel duct is a hexagonal shape matching that of the hex lattice boundary. OpenMC has the ability to represent arbitrarily oriented planes to model this duct.

In [ ]:
help(openmc.Plane)

However, a hexagonal shape is a common feature in reactor design and as such there is a convenience function in OpenMC for generating a hexagonal prism region.

In [ ]:
help(openmc.hexagonal_prism)

Now we can create the external duct and outer cells

Finally, we need an outer cell representing the space between assembly ducts.

Finally, let's make this a 3D model by adding upper and lower bounding planes.

We should now be able to use this as our geometry in the model and run it successfully.

---

# Advanced Tallies in OpenMC: Pincell Power Distributions



For this part of the exercise we're going to setup tallies that will provide us the pincell powers in the assembly we've created.

In [ ]:
# define ANL 33 group structure
ANL_33 = \
[1.000000E-05, 
4.174600E-01,
5.315800E-01,
3.927900E+00,
8.315300E+00,
1.371000E+01,
2.260300E+01,
3.726700E+01,
6.144200E+01,
1.013000E+02,
1.670200E+02,
2.753600E+02,
4.540000E+02,
7.485200E+02,
1.234100E+03,
2.034700E+03,
3.354600E+03,
5.530800E+03,
9.118800E+03,
1.503400E+04,
2.478700E+04,
4.086800E+04,
6.737900E+04,
1.110900E+05,
1.831600E+05,
3.019700E+05,
4.978700E+05,
8.208500E+05,
1.353400E+06,
2.231300E+06,
3.678800E+06,
6.065300E+06,
1.000000E+07,
1.419100E+07]

In [ ]:
import openmc.lib
def display_pin_powers(statepoint_file, pincell_tally, global_tally, origin):

    assembly_power = 65 / 79 * 1e6 # [W] (zero power test)

    with openmc.StatePoint(statepoint_file, autolink=False) as sp:
        pincell_tally = sp.get_tally(name=pincell_tally.name)
        tally_global_out = sp.get_tally(name=global_tally.name)
    
    total_kappa_fission = tally_global_out.get_values(scores=['kappa-fission']).flatten()[0]

    pincell_powers = pincell_tally.get_values(scores=['kappa-fission']).flatten()
    pincell_powers *= assembly_power / total_kappa_fission
    dcell_filter = pincell_tally.find_filter(openmc.DistribcellFilter)
    dcell = dcell_filter.bins[0]
    
    try:
        print('Initializing OpenMC...')
        openmc.lib.init(['-c'])
        print('Done')
        p = openmc.lib.plot._PlotBase()
        p.origin = origin
        p.width = 7
        p.height = 7
        p.h_res = 400
        p.v_res = 400
        p.colorby = 'cell'
        p.basis = 'xy'
        print('Generating id map...')
        id_mapping = openmc.lib.id_map(p)
        print('Done')
    finally:
        print('Shutting down OpenMC...')
        openmc.lib.finalize()
        print('Done')

    cell_map = id_mapping[:, :, 0]
    instance_map = id_mapping[:, :, 1]
    fuel_mask = cell_map == dcell
    power_mapping = np.zeros_like(cell_map, dtype=float)
    
    for i, h in enumerate(pincell_powers.flatten()):
        instance_mask = instance_map == i
        power_mapping[np.logical_and(fuel_mask, instance_mask)] = h

    power_mapping[power_mapping == 0.0] = np.nan
    
    plt.figure(figsize=(10,10))
    img = plt.imshow(power_mapping, extent=[-3.5, 3.5, -3.5, 3.5])
    plt.ylabel('Y')
    plt.xlabel('X')
    plt.colorbar(img, label='Power (W)')
    plt.show()

In [ ]:
display_pin_powers(statepoint, tally_pin_power, tally_global, (0.0, 0.0, 0.0))

In [ ]:
import random
random.choice(list(root_cell.region.get_surfaces().values())).boundary_type = 'vacuum'